# Construction et analyse des intervalles de confiance pour le SoH des tesla


In [ ]:
from transform.raw_results.config import *
from transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries
from core.pandas_utils import *
import plotly.graph_objects as go
import plotly.express as px


## data import

In [ ]:
def get_results():
    return (
        TeslaProcessedTimeSeries("tesla", filters=[("trimmed_in_charge", "==", True)])
        .groupby(["vin", "trimmed_in_charge_idx"], observed=True, as_index=False)
        .agg(
            energy_added_min=pd.NamedAgg("charge_energy_added", "min"),
            energy_added_end=pd.NamedAgg("charge_energy_added", "last"),
            soc_diff=pd.NamedAgg("soc", series_start_end_diff),
            inside_temp=pd.NamedAgg("inside_temp", "mean"),
            capacity=pd.NamedAgg("capacity", "first"),
            odometer=pd.NamedAgg("odometer", "first"),
            version=pd.NamedAgg("version", "first"),
            size=pd.NamedAgg("soc", "size"),
            model=pd.NamedAgg("model", "first"),
            date=pd.NamedAgg("date", "first"),
            charging_power=pd.NamedAgg("charging_power", "median"),
            tesla_code=pd.NamedAgg("tesla_code", "first"),
        )
        .eval("energy_added = energy_added_end - energy_added_min")
        .eval("soh = energy_added / (soc_diff / 100.0 * capacity)")
        .eval("level_1 = soc_diff * (charging_power < @LEVEL_1_MAX_POWER) / 100")
        .eval("level_2 = soc_diff * (charging_power.between(@LEVEL_1_MAX_POWER, @LEVEL_2_MAX_POWER)) / 100")
        .eval("level_3 = soc_diff * (charging_power > @LEVEL_2_MAX_POWER) / 100")
	    .eval("bottom_soh = soh.between(0.75, 0.9)")
        .eval("fixed_soh_min_end = soh.mask(tesla_code == 'MTY13', soh / 0.96)")
        .eval("fixed_soh_min_end = fixed_soh_min_end.mask(bottom_soh & tesla_code == 'MTY13', fixed_soh_min_end + 0.08)")
        .eval("soh = fixed_soh_min_end")
        .sort_values(["tesla_code", "vin", "date"])
    )

In [ ]:
df = get_results()

## IC build

In [ ]:
def ic_computation(df):
    len_ = df.shape[0]
    m = df['soh'].mean()
    std = df['soh'].std()
    if len_ > 0:
        upper_bound = m + 1.96 * (std/len_)
        lower_bound = m - 1.96 * (std/len_)
        
        return (round(lower_bound, 4),round( upper_bound, 4)), len_
    return (np.nan, np.nan), len_

In [ ]:
ic_df = pd.DataFrame(df.groupby(['vin'])[['soh']].apply(ic_computation))

In [ ]:
l1, number_charges = zip(*ic_df[0].values)
lower , upper = zip(*list(l1))

In [ ]:
ic_df['upper'] = list(upper)
ic_df['lower'] = list(lower)
ic_df["number_charges"] = list(number_charges)

In [ ]:
ic_df['ic_point_diff'] = ic_df['upper'] - ic_df['lower']

## Graphs

In [ ]:
hist_values, bin_edges = pd.cut(ic_df['ic_point_diff'], bins=[0, .01, .02, .03, .04, .05, .06, .07, .08, .09, .1], right=False, retbins=True)
hist_counts = hist_values.value_counts().sort_index()
fig = go.Figure(data=[go.Bar(
    x=[f"{round(bin_edges[i], 3)} - {round(bin_edges[i+1], 3)}" for i in range(len(bin_edges)-1)],
    y=hist_counts.values,
    marker=dict(color='blue'),
    text=hist_counts
)])
fig.update_layout(
    title="répartitions des tailles d'IC",
    xaxis_title="Intervale",
    yaxis_title="Frecuence"
)
fig.show()

In [ ]:
px.scatter(ic_df, x='number_charges', y='ic_point_diff', hover_data={'lower': True, 
                                                       'upper': True}, title='taille IC vs Nombre de charges')

In [ ]:
charges_df = ic_df.groupby("number_charges", as_index=False).agg(
    mean_ic_point = ('ic_point_diff', 'mean'),
    median_ic_point = ('ic_point_diff', 'median'),
    max_ic_point = ('ic_point_diff', 'max'),
    
).dropna()

In [ ]:
px.scatter(charges_df, x='number_charges', y='mean_ic_point', title='taille IC moyen par nombre de charges')

In [ ]:
px.scatter(charges_df, x='number_charges', y='median_ic_point', title='taille IC median par nombre de charges')

In [ ]:

px.scatter(charges_df, x='number_charges', y='max_ic_point', title = 'Plus grand IC par nombre de charges')